In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
x = torch.randn(10, 128, 512)

In [4]:
Q = torch.randn(128, 128)/(128)**0.5
W = torch.randn(128, 128)/(128)**0.5
E = torch.randn(128, 128)/(128)**0.5
## the above guys are attention guys

In [5]:
x_q = (Q @ x)
x_w = (W @ x)
x_e = (E @ x)

In [6]:
x_q.shape

torch.Size([10, 128, 512])

In [7]:
x_q = x_q.view(-1, 4, 32, 512)
x_w = x_w.view(-1, 4, 32, 512)
x_e = x_e.view(-1, 4, 32, 512)

In [8]:
(x_e @ nn.Softmax(-2)((x_q.transpose(-2, -1) @ x_w))).view(-1, 128, 512);

In [15]:
class multi_head_attention(nn.Module): 
    def __init__(self, embedding_dim = 128, heads = 4, lag = 512, dropout = 0.2, causal = True):
        super().__init__()
        
        assert (embedding_dim/heads).is_integer(), f"embedding_dim/heads should be integer while yours {embedding_dim/heads} "
        
        self.embedding_dim = embedding_dim
        self.heads = heads 
        self.causal = causal
        self.W = lag
        ### Attention Part ###
        ### Frist Dropout layers
        self.dropout_Q = nn.Dropout(p = dropout)
        self.dropout_K = nn.Dropout(p = dropout)
        self.dropout_V = nn.Dropout(p = dropout)
        ### Weights ###
        self.L_Q = nn.Parameter(torch.randn(embedding_dim, embedding_dim)*(embedding_dim)**(-0.5))
        self.L_K = nn.Parameter(torch.randn(embedding_dim, embedding_dim)*(embedding_dim)**(-0.5))
        self.L_V = nn.Parameter(torch.randn(embedding_dim, embedding_dim)*(embedding_dim)**(-0.5))
        ### --- End of weights --- ###
        if self.causal:
            self.causal_factor = nn.Parameter(torch.tril(-torch.inf*torch.ones(self.W,self.W), diagonal = -1))
            ## No gradient is required in the above layer....
            self.causal_factor.requires_grad = False
    
    def forward(self, x): #BxHxL -> BxHxL
        K, Q, V = x[0], x[1], x[2]
        K = self.dropout_K(self.L_K @ K)
        Q = self.dropout_Q(self.L_Q @ Q)
        V = self.dropout_V(self.L_V @ V)
        return K, Q, V

In [26]:
x = multi_head_attention(128, heads = 4)